In [3]:
#첨부파일 처리 - 파일 복사 및 json path 변경

import json
import os
import shutil
from pathlib import Path

# ==================== 설정 영역 (여기를 수정하세요) ====================

# 첨부파일 원본 경로들 - 여러 경로 추가 가능
BASE_FILE_PATHS = [
    r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\file\brd",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\file\brd",
    r"C:\Users\LEEJUHWAN\Downloads\2011-01-01~2015-12-31\html\resource\file\brd",
    r"C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html\resource\file\brd",
    # 더 추가하고 싶으면 여기에 계속 추가

]

# 첨부파일 복사본 저장 폴더
OUTPUT_FILE_FOLDER = r"C:\Users\LEEJUHWAN\Downloads\board_qna_attachments"

# 처리할 JSON 파일
JSON_FILE = r"board_posts_imgConvert.json"

# path에서 제거할 접두사 (이 부분을 제거하고 상대경로로 변경)
REMOVE_PREFIX = r"C:\Users\LEEJUHWAN\Downloads"

# ======================================================================


def find_attachment_folder(board_id):
    """여러 경로에서 게시판 ID에 해당하는 첨부파일 폴더 찾기"""
    for base_path in BASE_FILE_PATHS:
        attachment_folder = Path(base_path) / board_id
        if attachment_folder.exists():
            return attachment_folder
    return None


def find_file_in_folder(folder, filename):
    """폴더 내에서 파일명과 일치하는 파일 찾기 (대소문자 구분 없음)"""
    if not folder or not folder.exists():
        return None
    
    # 정확한 파일명으로 먼저 찾기
    target_file = folder / filename
    if target_file.exists() and target_file.is_file():
        return target_file
    
    # 대소문자 구분 없이 찾기
    for file in folder.iterdir():
        if file.is_file() and file.name.lower() == filename.lower():
            return file
    
    return None


def process_single_post_attachments(data, post_index):
    """하나의 post의 첨부파일 처리"""
    
    # sourceId에서 게시판ID 추출
    source_id = data.get('sourceId', '')
    if not source_id:
        print(f"⚠️ [{post_index}] sourceId 없음, 건너뜀")
        return data
    
    board_id = source_id.lstrip('_')
    
    # attaches 배열 확인
    attaches = data.get('attaches', [])
    if not attaches:
        return data  # 첨부파일 없으면 그대로 반환
    
    # 여러 경로에서 첨부파일 폴더 찾기
    attachment_folder = find_attachment_folder(board_id)
    
    if not attachment_folder:
        print(f"⚠️ [{post_index}] 게시판ID {board_id} 폴더 없음 (모든 경로 검색 완료)")
        # 폴더가 없으면 모든 첨부파일의 path를 공란으로
        for attach in attaches:
            attach['path'] = ''
        return data
    
    # 복사본 저장할 게시판별 폴더 생성
    output_board_folder = Path(OUTPUT_FILE_FOLDER) / f"brd{board_id}"
    output_board_folder.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📄 [{post_index}] 게시판ID: {board_id}")
    print(f"   원본: {attachment_folder}")
    print(f"   복사: {output_board_folder}")
    print(f"   ✅ 첨부파일 {len(attaches)}개 발견")
    
    # 각 첨부파일 처리
    processed_attaches = []
    copied_count = 0
    
    for idx, attach in enumerate(attaches):
        filename = attach.get('name', '')
        if not filename:
            print(f"      ⚠️ [{idx}] 파일명 없음 - path 공란 처리")
            attach['path'] = ''
            processed_attaches.append(attach)
            continue
        
        # 원본 폴더에서 파일 찾기
        source_file = find_file_in_folder(attachment_folder, filename)
        
        if source_file:
            # 복사본 파일 경로
            dest_file = output_board_folder / filename
            
            # 파일 복사
            try:
                shutil.copy2(source_file, dest_file)
                copied_count += 1
                print(f"      ✅ {filename}")
                
                # path를 상대경로로 변경 (REMOVE_PREFIX 제거)
                new_path = str(dest_file).replace('\\', '/')
                if REMOVE_PREFIX:
                    new_path = new_path.replace(REMOVE_PREFIX.replace('\\', '/'), '').lstrip('/')
                    new_path = new_path.replace(REMOVE_PREFIX, '').lstrip('\\')
                
                attach['path'] = new_path
                
            except Exception as e:
                print(f"      ❌ {filename} 복사 실패: {e}")
                attach['path'] = ''
        else:
            print(f"      ⚠️ 파일 없음: {filename} - path 공란 처리")
            attach['path'] = ''
        
        processed_attaches.append(attach)
    
    print(f"   📊 복사 완료: {copied_count}/{len(attaches)}개")
    
    # 수정된 attaches 적용
    data['attaches'] = processed_attaches
    return data


def process_json_file(json_file_path):
    """JSON 파일 처리"""
    
    # 출력 폴더 생성
    os.makedirs(OUTPUT_FILE_FOLDER, exist_ok=True)
    
    # 1. JSON 파일 읽기
    print(f"📂 파일 읽는 중: {json_file_path}")
    print(f"📂 검색할 첨부파일 경로: {len(BASE_FILE_PATHS)}개\n")
    
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    posts = data.get('posts', [])
    total_count = len(posts)
    
    print(f"✅ 총 {total_count}개의 posts 발견\n")
    
    # 2. 각 post 처리
    processed_posts = []
    success_count = 0
    total_attachments = 0
    
    for idx, post in enumerate(posts, 1):
        try:
            # 첨부파일이 있는 경우만 카운트
            if post.get('attaches'):
                total_attachments += len(post.get('attaches'))
            
            result = process_single_post_attachments(post, idx)
            processed_posts.append(result)
            
            if result != post:  # 변경사항이 있으면
                success_count += 1
        except Exception as e:
            print(f"❌ [Post {idx}] 처리 오류: {e}")
            processed_posts.append(post)  # 오류 시 원본 유지
            continue
    
    # 3. 수정된 데이터 저장
    output_file = json_file_path.replace('.json', '_attachConvert.json')
    
    output_data = {"posts": processed_posts}
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n{'='*60}")
    print(f"✅ 처리 완료!")
    print(f"   - 첨부파일 있는 posts: {success_count}개")
    print(f"   - 총 첨부파일 개수: {total_attachments}개")
    print(f"   - 복사된 파일 위치: {OUTPUT_FILE_FOLDER}")
    print(f"   - 수정된 JSON 파일: {output_file}")
    print(f"{'='*60}")


# 실행
if __name__ == "__main__":
    process_json_file(JSON_FILE)

📂 파일 읽는 중: board_posts_imgConvert.json
📂 검색할 첨부파일 경로: 4개

✅ 총 79개의 posts 발견


📄 [3] 게시판ID: 2614054
   원본: C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\file\brd\2614054
   복사: C:\Users\LEEJUHWAN\Downloads\board_qna_attachments\brd2614054
   ✅ 첨부파일 1개 발견
      ✅ 2019_bitrends_magazine_ko-kr.pdf
   📊 복사 완료: 1/1개

📄 [5] 게시판ID: 2609354
   원본: C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\file\brd\2609354
   복사: C:\Users\LEEJUHWAN\Downloads\board_qna_attachments\brd2609354
   ✅ 첨부파일 1개 발견
      ✅ XXXGridDataHandler.java
   📊 복사 완료: 1/1개

📄 [6] 게시판ID: 2541843
   원본: C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\file\brd\2541843
   복사: C:\Users\LEEJUHWAN\Downloads\board_qna_attachments\brd2541843
   ✅ 첨부파일 2개 발견
      ✅ NCD_Platform소개자료_M_v2.0.pdf
      ✅ LG CNS EAP 설명회_20180917.pdf
   📊 복사 완료: 2/2개

📄 [8] 게시판ID: 2420619
   원본: C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\file\brd\2420619
   복사: C:\Users\LEEJUHWAN\Do